In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,Sentence,Label
0,0,The Cardiff Roller Collective (CRoC) are a rol...,11
1,1,"""Go! Pack Go!"" is the fight song of the Green ...",11
2,2,Al-Machriq (English translation: The East) was...,14
3,3,Ajman International Airport (Arabic: مطار عجما...,0
4,4,Kapla is a construction set for children and a...,4


In [5]:
from transformers import AutoTokenizer, AutoModel
import transformers
import torch
import numpy as np

In [4]:
model = "meta-llama/Llama-2-7b-chat-hf"

# meta-llama/Meta-Llama-3-8B

tokenizer = AutoTokenizer.from_pretrained(model)

In [6]:
pipeline = transformers.pipeline("feature-extraction", model=model,torch_dtype=torch.float32)

embeds = pipeline(df['Sentence'].tolist(), tokenizer=tokenizer, eos_token=tokenizer.eos_token, padding=True, truncation=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def get_embedding(text, chunk_size, model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=False)

    # Split tokens to chunks
    chunks = [tokens[x:x+chunk_size] for x in range(0, len(tokens), chunk_size)]

    # Get embeddings for all chunks
    embeddings = []
    for chunk in chunks:
        with torch.no_grad():
            outputs = model(input_ids=chunk.unsqueeze(0))
            chunk_embeddings = outputs.last_hidden_state.squeeze(0)
            embeddings.append(chunk_embeddings)

    # Concatenate all chunks to get the whole sentence embedding
    embeddings = torch.cat(embeddings, dim=0)

    return embeddings


print(get_embedding(df['Sentence'].tolist(), 10, model).shape)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]